## Import extreme multi-label dataset
- in extreme-sparse dataset, test wide n deep 

In [1]:
import os

import pandas as pd
import numpy as np
from scipy.io import loadmat

/home/admin_user/anaconda3/envs/gsshop/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/admin_user/anaconda3/envs/gsshop/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
dataset="bibtex"
data_dir='data/{}'.format(dataset)
print(data_dir)
def load_input():
    data = list(loadmat(data_dir + '/input.mat')['data'][0][0])
    return data[:4]

data/bibtex


In [3]:
train_X, train_Y, test_X, test_Y = load_input()

In [4]:
train_X = train_X.toarray()
train_Y = train_Y.toarray()
test_X = test_X.toarray()
test_Y = test_Y.toarray()

print(train_X.shape)
print(train_Y.shape)

(4880, 1836)
(4880, 159)


In [5]:
pd.DataFrame(train_X).head()

,0,1,2,3,4,5,6,7,8,9,...,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
extreme_x_train = np.concatenate((train_X, train_X), axis=1)
extreme_x_train = np.concatenate((extreme_x_train, extreme_x_train), axis=1)
extreme_x_train = np.concatenate((extreme_x_train, extreme_x_train), axis=1)
extreme_x_train.shape

(4880, 3672)

In [8]:
extreme_x_test = np.concatenate((test_X, test_X), axis=1)
extreme_x_test = np.concatenate((extreme_x_test, extreme_x_test), axis=1)
extreme_x_test = np.concatenate((extreme_x_test, extreme_x_test), axis=1)
extreme_x_test.shape

(2515, 3672)

In [9]:
extreme_y_train = np.concatenate((train_Y, train_Y), axis=1)
extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
extreme_y_train.shape

(4880, 10176)

In [12]:
extreme_y_test = np.concatenate((test_Y, test_Y), axis=1)
extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
extreme_y_test.shape

(2515, 10176)

## Coding wide-n-deep with keras

In [ ]:
from scipy.io import loadmat

from keras.models import Sequential, Model
from keras.layers import Input, Dense, concatenate
from keras.optimizers import Adam


# dataset
dataset = "bibtex"
data_dir = 'data/{}'.format(dataset)

def load_input():
    data = list(loadmat(data_dir + '/input.mat')['data'][0][0])
    return data[:4]


class Deep:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(5000, activation='relu', input_dim=self.input_dim))
        model.add(Dense(2500, activation='relu'))
        model.add(Dense(500, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self.output_dim, activation='softmax'))

        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


class Wide:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        model = Sequential()
        model.add(Dense(self.output_dim, activation='softmax', input_dim=self.input_dim))

        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, x, y):
        self.model.fit(x, y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, x, y):
        performance_test = self.model.evaluate(x, y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


class WideAndDeep:
    def __init__(self, batch_size, epochs, learning_rate, input_dim, output_dim):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = self.classifier()

    def classifier(self):
        optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)

        # wide part
        wide = Input(shape=(self.input_dim,))

        # deep part
        deep_input = Input(shape=(self.input_dim,))
        deep = Dense(5000, activation='relu')(deep_input)
        deep = Dense(2500, activation='relu')(deep)
        deep = Dense(500, activation='relu')(deep)
        deep = Dense(50, activation='relu')(deep)

        # concatenate : wide and deep
        wide_n_deep = concatenate([wide, deep])
        wide_n_deep = Dense(self.output_dim, activation='softmax')(wide_n_deep)
        model = Model(inputs=[wide, deep_input], outputs=wide_n_deep)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
        return model

    def fit(self, wide_x, deep_x, y):
        self.model.fit([wide_x, deep_x], y, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.2)

    def print_performance(self, wide_x, deep_x, y):
        performance_test = self.model.evaluate([wide_x, deep_x], y, batch_size=self.batch_size)
        print('Test Loss and Accuracy ->', performance_test)


def main(model_param):
    # prepare dataset
    x_train, y_train, x_test, y_test = load_input()
    x_train = x_train.toarray()
    y_train = y_train.toarray()
    x_test = x_test.toarray()
    y_test = y_test.toarray()

    # extreme test
    extreme_x_train = np.concatenate((x_train, x_train), axis=1)
    extreme_x_train = np.concatenate((extreme_x_train, extreme_x_train), axis=1)
    extreme_x_train = np.concatenate((extreme_x_train, extreme_x_train), axis=1)
    extreme_x_test = np.concatenate((x_test, x_test), axis=1)
    extreme_x_test = np.concatenate((extreme_x_test, extreme_x_test), axis=1)
    extreme_x_test = np.concatenate((extreme_x_test, extreme_x_test), axis=1)
    extreme_y_train = np.concatenate((y_train, y_train), axis=1)
    extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
    extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
    extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
    extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
    extreme_y_train = np.concatenate((extreme_y_train, extreme_y_train), axis=1)
    extreme_y_test = np.concatenate((y_test, y_test), axis=1)
    extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
    extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
    extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
    extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)
    extreme_y_test = np.concatenate((extreme_y_test, extreme_y_test), axis=1)

    x_train = extreme_x_train
    y_train = extreme_y_train
    x_test = extreme_x_test
    y_test = extreme_y_test

    # prepare hyper parameter
    batch_size = 500
    epochs = 10
    learning_rate = 0.001
    input_dim = x_train.shape[1]
    output_dim = y_train.shape[1]
    

    if model_param == "deep":
        deep = Deep(batch_size, epochs, learning_rate, input_dim, output_dim)
        deep.fit(x_train, y_train)
        deep.print_performance(x_test, y_test)
    elif model_param == 'wide':
        wide = Wide(batch_size, epochs, learning_rate, input_dim, output_dim)
        wide.fit(x_train, y_train)
        wide.print_performance(x_test, y_test)
    else:
        wide_n_deep = WideAndDeep(batch_size, epochs, learning_rate, input_dim, output_dim)
        wide_n_deep.fit(x_train, x_train, y_train)
        wide_n_deep.print_performance(x_test, x_test, y_test)

        # prediction for individual and y_column rank
        x_predict_test = x_test[np.newaxis, 0, :]
        y_predict_test = y_test[0]
        result = wide_n_deep.model.predict([x_predict_test, x_predict_test])
        print('result predicted:', result)
        print('result real:', y_predict_test)

        # select top 10 y's column index in result(softmax prediction)
        top_10_y_column = result[0].argsort()[-10:][::-1].tolist()
        print('result top 10:', top_10_y_column)


if __name__ == '__main__':
    main('deep')

Train on 3904 samples, validate on 976 samples
Epoch 1/10
3904/3904 [==============================] - 22s 6ms/step - loss: 1398.1033 - acc: 0.0000e+00 - val_loss: 1407.1631 - val_acc: 0.0000e+00
Epoch 2/10
3904/3904 [==============================] - 18s 5ms/step - loss: 1329.0627 - acc: 0.0036 - val_loss: 1335.6464 - val_acc: 0.0041
Epoch 3/10
3904/3904 [==============================] - 18s 5ms/step - loss: 1258.5543 - acc: 0.0020 - val_loss: 1291.5097 - val_acc: 0.0000e+00
Epoch 4/10
3904/3904 [==============================] - 18s 5ms/step - loss: 1208.5334 - acc: 7.6844e-04 - val_loss: 1258.6562 - val_acc: 0.0000e+00
Epoch 5/10
3904/3904 [==============================] - 18s 5ms/step - loss: 1161.7479 - acc: 0.0015 - val_loss: 1234.7146 - val_acc: 0.0000e+00
Epoch 6/10
3904/3904 [==============================] - 18s 5ms/step - loss: 1124.3823 - acc: 0.0018 - val_loss: 1238.5021 - val_acc: 0.0000e+00
Epoch 7/10
3904/3904 [==============================] - 18s 5ms/step - loss: 10

## result
- extreme sparse classification is working
- but wide and deep is too slow
- deep, wide is little slow
- so, X features must be reduction